In [13]:
import os

import numpy as np
import pandas as pd
from scipy.stats import median_absolute_deviation
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import optuna
from sklearn.preprocessing import StandardScaler
import random
random_state = 1234
from sklearn.metrics import roc_auc_score
import os
from pathlib import Path

In [3]:
parent_dir = Path(os.getcwd()).parent
cross_validation_dir = os.path.join(parent_dir, "Data", "train_test_indices.npy")
data_dir = os.path.join(parent_dir, "Data" , "MSK_Impact_train")
train_test_indices = np.load(cross_validation_dir, allow_pickle = True)

In [4]:
def list_files_in_dir(dirname):
    dir_files = list()
    for root, _, files in os.walk(dirname):
        for file in files:
            dir_files.append(os.path.join(root, file))    
    return dir_files

In [5]:
def build_kneigh_model(n_neighbors, X, y):
    model = KNeighborsClassifier(n_neighbors=n_neighbors)
    model.fit(X, y)
    
    return model

In [6]:
def scale_rows(X):
    return X.div(X.sum(axis=1), axis=0)

In [7]:
def get_row_indices_with_sum_zero(X):
    return X.index[(X.sum(axis=1) == 0)].tolist()

In [8]:
def dataset_generator(data_dir, y_col):

    for datafile in list_files_in_dir(data_dir):
        data = pd.read_csv(datafile)
        X = data.iloc[:, :96]
        y = (data[y_col]).astype(np.int_)
        X.columns = X.columns.str.replace('[', '').str.replace(']', '').str.replace('>', '')
        X = scale_rows(X)
        yield X, y

In [ ]:
def train_val_test_generator(data_dir, num_samples, y_col, test_frac=0.1, n_folds = 10):
    val_frac = test_frac/(1.0 - test_frac)
    
    for X, y in dataset_generator(data_dir, num_samples, y_col):
        fold_data = list()
        skf = StratifiedKFold(n_splits=n_folds, shuffle = True, random_state = random_state)
#         print(random_state)
#         skf = StratifiedKFold(n_splits=num_folds)
        
        for train_val_index, test_index in skf.split(X, y):
            X_test, y_test = X.iloc[test_index, :], y.iloc[test_index]
            X_train, X_val, y_train, y_val = train_test_split(
                X.iloc[train_val_index,:], y.iloc[train_val_index], 
                test_size=val_frac, 
                random_state=random_state, 
                stratify=y.iloc[train_val_index]
            )
#             print(train_val_index, test_index)
            
            #X_train, X_val, X_test = scale_columns(X_train, X_val, X_test)
            fold_data.append(((X_train, y_train), 
                              (X_val, y_val), 
                              (X_test, y_test)))
            
        yield fold_data

In [9]:
def train_val_test_generator_default(data_dir, y_col, test_frac=0.1):
    
    val_frac = test_frac/(1.0 - test_frac)
    
    for X, y in dataset_generator(data_dir, y_col):
        fold_data = list()

        
        for train_val_index, test_index in train_test_indices:
            X_test, y_test = X.iloc[test_index, :], y.iloc[test_index]
            X_train, X_val, y_train, y_val = train_test_split(
                X.iloc[train_val_index,:], y.iloc[train_val_index], 
                test_size=val_frac, 
                random_state=random_state, 
                stratify=y.iloc[train_val_index]
            )
#             print(train_val_index, test_index)
            
            #X_train, X_val, X_test = scale_columns(X_train, X_val, X_test)
            fold_data.append(((X_train, y_train), 
                              (X_val, y_val), 
                              (X_test, y_test)))
            
        yield fold_data

In [27]:
def compute_mlp_performance(trial, data_dir=data_dir, n_neighbors=5, n_folds=10, y_col='is_sig3_20'):
    aucs = list()

    for folds_data in train_val_test_generator_default(data_dir, y_col=y_col):
        fold_aucs = list()
        
        for fold_data in folds_data:
            # get data
            (X_train, y_train),(X_val, y_val), (X_test, y_test) = fold_data

            model = build_kneigh_model(trial.suggest_int('n_neighbors', 1, 100), X_train, y_train)

            # evaluate
            fold_aucs.append(roc_auc_score(y_test, model.predict(X_test)))
            
        aucs.append(fold_aucs)
    
    aucs = np.array(aucs)
    
    median_of_median_auc = np.median(np.median(aucs, axis=1))
    mad_of_mad_auc = median_absolute_deviation(median_absolute_deviation(aucs, axis=1))
#     return median_of_median_auc, median_absolute_deviation(aucs, axis=1)
    return median_of_median_auc

    

In [28]:
study_is_sig3 = optuna.create_study(direction="maximize")
study_is_sig3.optimize(compute_mlp_performance, n_trials=200)

[I 2022-06-27 13:51:36,973] A new study created in memory with name: no-name-017c0571-2924-4596-81fb-21e12c627d34
C:\Users\aaron\AppData\Local\Temp\ipykernel_4840\2323069173.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  X.columns = X.columns.str.replace('[', '').str.replace(']', '').str.replace('>', '')
C:\Users\aaron\AppData\Local\Temp\ipykernel_4840\2323069173.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  X.columns = X.columns.str.replace('[', '').str.replace(']', '').str.replace('>', '')
C:\Users\aaron\AppData\Local\Temp\ipykernel_4840\925004748.py:23: DeprecationWarning: `median_absolute_deviation` is deprecated, use `median_abs_deviation` instead!

To preserv

C:\Users\aaron\AppData\Local\Temp\ipykernel_4840\2323069173.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  X.columns = X.columns.str.replace('[', '').str.replace(']', '').str.replace('>', '')
C:\Users\aaron\AppData\Local\Temp\ipykernel_4840\2323069173.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  X.columns = X.columns.str.replace('[', '').str.replace(']', '').str.replace('>', '')
C:\Users\aaron\AppData\Local\Temp\ipykernel_4840\925004748.py:23: DeprecationWarning: `median_absolute_deviation` is deprecated, use `median_abs_deviation` instead!

To preserve the existing default behavior, use
`scipy.stats.median_abs_deviation(..., scale=1/1.4826)`.
The value 1.4826 is 

C:\Users\aaron\AppData\Local\Temp\ipykernel_4840\2323069173.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  X.columns = X.columns.str.replace('[', '').str.replace(']', '').str.replace('>', '')
C:\Users\aaron\AppData\Local\Temp\ipykernel_4840\2323069173.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  X.columns = X.columns.str.replace('[', '').str.replace(']', '').str.replace('>', '')
C:\Users\aaron\AppData\Local\Temp\ipykernel_4840\925004748.py:23: DeprecationWarning: `median_absolute_deviation` is deprecated, use `median_abs_deviation` instead!

To preserve the existing default behavior, use
`scipy.stats.median_abs_deviation(..., scale=1/1.4826)`.
The value 1.4826 is 

C:\Users\aaron\AppData\Local\Temp\ipykernel_4840\2323069173.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  X.columns = X.columns.str.replace('[', '').str.replace(']', '').str.replace('>', '')
C:\Users\aaron\AppData\Local\Temp\ipykernel_4840\2323069173.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  X.columns = X.columns.str.replace('[', '').str.replace(']', '').str.replace('>', '')
C:\Users\aaron\AppData\Local\Temp\ipykernel_4840\925004748.py:23: DeprecationWarning: `median_absolute_deviation` is deprecated, use `median_abs_deviation` instead!

To preserve the existing default behavior, use
`scipy.stats.median_abs_deviation(..., scale=1/1.4826)`.
The value 1.4826 is 

C:\Users\aaron\AppData\Local\Temp\ipykernel_4840\2323069173.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  X.columns = X.columns.str.replace('[', '').str.replace(']', '').str.replace('>', '')
C:\Users\aaron\AppData\Local\Temp\ipykernel_4840\2323069173.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  X.columns = X.columns.str.replace('[', '').str.replace(']', '').str.replace('>', '')
C:\Users\aaron\AppData\Local\Temp\ipykernel_4840\925004748.py:23: DeprecationWarning: `median_absolute_deviation` is deprecated, use `median_abs_deviation` instead!

To preserve the existing default behavior, use
`scipy.stats.median_abs_deviation(..., scale=1/1.4826)`.
The value 1.4826 is 

...

C:\Users\aaron\AppData\Local\Temp\ipykernel_4840\2323069173.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  X.columns = X.columns.str.replace('[', '').str.replace(']', '').str.replace('>', '')
C:\Users\aaron\AppData\Local\Temp\ipykernel_4840\2323069173.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  X.columns = X.columns.str.replace('[', '').str.replace(']', '').str.replace('>', '')
C:\Users\aaron\AppData\Local\Temp\ipykernel_4840\925004748.py:23: DeprecationWarning: `median_absolute_deviation` is deprecated, use `median_abs_deviation` instead!

To preserve the existing default behavior, use
`scipy.stats.median_abs_deviation(..., scale=1/1.4826)`.
The value 1.4826 is 

C:\Users\aaron\AppData\Local\Temp\ipykernel_4840\2323069173.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  X.columns = X.columns.str.replace('[', '').str.replace(']', '').str.replace('>', '')
C:\Users\aaron\AppData\Local\Temp\ipykernel_4840\2323069173.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  X.columns = X.columns.str.replace('[', '').str.replace(']', '').str.replace('>', '')
C:\Users\aaron\AppData\Local\Temp\ipykernel_4840\925004748.py:23: DeprecationWarning: `median_absolute_deviation` is deprecated, use `median_abs_deviation` instead!

To preserve the existing default behavior, use
`scipy.stats.median_abs_deviation(..., scale=1/1.4826)`.
The value 1.4826 is 

C:\Users\aaron\AppData\Local\Temp\ipykernel_4840\2323069173.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  X.columns = X.columns.str.replace('[', '').str.replace(']', '').str.replace('>', '')
C:\Users\aaron\AppData\Local\Temp\ipykernel_4840\2323069173.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  X.columns = X.columns.str.replace('[', '').str.replace(']', '').str.replace('>', '')
C:\Users\aaron\AppData\Local\Temp\ipykernel_4840\925004748.py:23: DeprecationWarning: `median_absolute_deviation` is deprecated, use `median_abs_deviation` instead!

To preserve the existing default behavior, use
`scipy.stats.median_abs_deviation(..., scale=1/1.4826)`.
The value 1.4826 is 

C:\Users\aaron\AppData\Local\Temp\ipykernel_4840\2323069173.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  X.columns = X.columns.str.replace('[', '').str.replace(']', '').str.replace('>', '')
C:\Users\aaron\AppData\Local\Temp\ipykernel_4840\2323069173.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  X.columns = X.columns.str.replace('[', '').str.replace(']', '').str.replace('>', '')
C:\Users\aaron\AppData\Local\Temp\ipykernel_4840\925004748.py:23: DeprecationWarning: `median_absolute_deviation` is deprecated, use `median_abs_deviation` instead!

To preserve the existing default behavior, use
`scipy.stats.median_abs_deviation(..., scale=1/1.4826)`.
The value 1.4826 is 

C:\Users\aaron\AppData\Local\Temp\ipykernel_4840\2323069173.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  X.columns = X.columns.str.replace('[', '').str.replace(']', '').str.replace('>', '')
C:\Users\aaron\AppData\Local\Temp\ipykernel_4840\2323069173.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  X.columns = X.columns.str.replace('[', '').str.replace(']', '').str.replace('>', '')
C:\Users\aaron\AppData\Local\Temp\ipykernel_4840\925004748.py:23: DeprecationWarning: `median_absolute_deviation` is deprecated, use `median_abs_deviation` instead!

To preserve the existing default behavior, use
`scipy.stats.median_abs_deviation(..., scale=1/1.4826)`.
The value 1.4826 is 

In [24]:
print(study_is_sig3.best_trial)

FrozenTrial(number=12, values=[0.6110182088442958], datetime_start=datetime.datetime(2022, 6, 27, 13, 48, 19, 990363), datetime_complete=datetime.datetime(2022, 6, 27, 13, 48, 20, 140959), params={'n_neighbors': 1}, distributions={'n_neighbors': IntUniformDistribution(high=100, low=1, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=12, state=TrialState.COMPLETE, value=None)


In [21]:
!conda env export | grep -v "^prefix: " > environment.yml